# 1: Corpora and Vector Spaces

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "/tmp" will be used to save temporary dictionary and corpus.


to see logging events.

## From Strings to Vectors

start from documents represented as strings:

In [0]:
! pip install gensim

    100% |████████████████████████████████| 22.6MB 1.2MB/s 
    100% |████████████████████████████████| 1.4MB 10.2MB/s 
    100% |████████████████████████████████| 133kB 22.2MB/s 
    100% |████████████████████████████████| 4.2MB 4.8MB/s 
    100% |████████████████████████████████| 61kB 14.5MB/s 
    100% |████████████████████████████████| 552kB 14.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /content/.cache/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
from gensim import corpora

2018-05-12 06:23:08,578 : INFO : 'pattern' package not found; tag filters are not available for English


In [0]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

This is a tiny corpus of nine documents, each consisting of only a single sentence. (array...)

BUT First,  tokenize the documents, remove common words (using a toy stoplist)   Stom list is a fancy way of saying stuff we want to remove... from the corpus

as well as words that only appear once in the corpus:

In [0]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict # fancy high performance data structure
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts] # cool list comprehencsion again :)

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


Your way of processing the documents will likely vary; here, I only split on whitespace to tokenize, followed by lowercasing each word. In fact, I use this particular (simplistic and inefficient) setup to mimic the experiment done in [Deerwester et al.’s original LSA article](http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf) (Table 2).

The ways to process documents are so varied and application- and language-dependent that I decided to not constrain them by any interface. Instead, a document is represented by the features extracted from it, not by its “surface” string form: how you get to the features is up to you. Below I describe one common, general-purpose approach (called bag-of-words), but keep in mind that different application domains call for different features, and, as always, it’s [garbage in, garbage out](https://en.wikipedia.org/wiki/Garbage_in,_garbage_out)...

To convert documents to vectors, we’ll use a document representation called [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model). In this representation, each document is represented by one vector where a vector element `i` represents the number of times the `i`th word appears in the document.

It is advantageous to represent the questions only by their (integer) ids. The mapping between the questions and ids is called a dictionary:

In [0]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)

2018-05-12 06:23:11,158 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-12 06:23:11,169 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)
2018-05-12 06:23:11,174 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2018-05-12 06:23:11,177 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


Here we assigned a unique integer ID to all words appearing in the processed corpus with the [gensim.corpora.dictionary.Dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary) class. This sweeps across the texts, collecting word counts and relevant statistics. In the end, we see there are twelve distinct words in the processed corpus, which means each document will be represented by twelve numbers (ie., by a 12-D vector). To see the mapping between words and their ids:

In [0]:
print(dictionary.token2id)

{u'minors': 11, u'graph': 10, u'system': 5, u'trees': 9, u'eps': 8, u'computer': 0, u'survey': 4, u'user': 7, u'human': 1, u'time': 6, u'interface': 2, u'response': 3}


To actually convert tokenized documents to vectors:

In [0]:
new_doc = "Human human computer interaction system"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored 

[(0, 1), (1, 2), (5, 1)]


The function `doc2bow()` simply counts the number of occurrences of each distinct word,   
converts the word to its integer word id and returns the result as a bag-of-words--a sparse vector, in the form of  
`[(word_id, word_count), ...]`. 

As the token_id is 0 for *"human"* and 2 for *"computer"*,   
the new document *“Human computer interaction”* will be transformed to  
[(0, 1), (2, 1)].   

*actually 1 is for human and 0 for computer in this run of the code..* not sure y  

The words *"computer"* and *"system"* exist in the dictionary and appear once. Thus, they become (0, 1),  (5, 1)respectively in the sparse vector.   

we added another human just to confrimt he mapping as seen above 


The word *"interaction"* doesn't exist in the dictionary and, thus, will not show up in the sparse vector. The other ten dictionary words, that appear (implicitly) zero times, will not show up in the sparse vector and , ,there will never be a element in the sparse vector like (3, 0).

For people familiar with scikit learn, `doc2bow()` has similar behaviors as calling `transform()` on [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). `doc2bow()` can behave like `fit_transform()` as well. For more details, please look at [gensim API Doc](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow).

In [0]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
for c in corpus:
    print(c)

2018-05-12 06:23:14,025 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2018-05-12 06:23:14,028 : INFO : saving sparse matrix to /tmp/deerwester.mm
2018-05-12 06:23:14,033 : INFO : PROGRESS: saving document #0
2018-05-12 06:23:14,039 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2018-05-12 06:23:14,040 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


By now it should be clear that the vector feature with `id=10` represents the number of times the word "graph" occurs in the document.  The answer is “zero” for the first six documents and “one” for the remaining three. As a matter of fact, we have arrived at exactly the same corpus of vectors as in the [Quick Example](https://radimrehurek.com/gensim/tutorial.html#first-example). If you're running this notebook yourself the word IDs may differ, but you should be able to check the consistency between documents comparing their vectors. 

## Corpus Streaming – One Document at a Time

Note that *corpus* above resides fully in memory, as a plain Python list. In this simple example, it doesn’t matter much, but just to make things clear, let’s assume there are millions of documents in the corpus.   
Storing all of them in RAM won’t do. Instead, let’s assume the documents are stored in a file on disk, one document per line. Gensim only requires that a corpus be able to return one document (the document mentioned just now being one line in the actual document)vector at a time:

In [0]:
#when you get lazy to upload a file you do all kinds of jugaad
! pip install numpy

In [0]:
import numpy as np
#! touch datasets/mycorpus.txt

In [0]:
np.savetxt('mycorpus.txt', documents,  fmt="%s")

In [0]:
from smart_open import smart_open
class MyCorpus(object):
    def __iter__(self):
        for line in smart_open('mycorpus.txt', 'rb'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

The assumption that each document occupies one line in a single file is not important; you can design the `__iter__` function to fit your input format, whatever that may be - walking directories, parsing XML, accessing network nodes... Just parse your input to retrieve a clean list of tokens in each document, then convert the tokens via a dictionary to their IDs and yield the resulting sparse vector inside `__iter__`.

In [0]:
corpus_memory_friendly = MyCorpus() # doesn't load the corpus into memory!
print(corpus_memory_friendly)

`corpus_memory_friendly` is now an object. We didn’t define any way to print it, so `print` just outputs address of the object in memory. Not very useful. To see the constituent vectors, let’s iterate over the corpus and print each document vector (one at a time):

In [0]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


Although the output is the same as for the plain Python list, the corpus is now much more memory friendly, because at most one vector resides in RAM at a time. Your corpus can now be as large as you want.

We are going to create the dictionary from the mycorpus.txt file without loading the entire file into memory.   
Then, we will generate the list of token ids to remove from this dictionary by querying the dictionary for the token ids of the stop words,   
and by querying the document frequencies dictionary (`dictionary.dfs`) for token ids that only appear once.  

Finally, we will filter these token ids out of our dictionary.   
Keep in mind that `dictionary.filter_tokens` (and some other functions such as `dictionary.add_document`)   
will call `dictionary.compactify()` to remove the gaps in the token id series thus enumeration of remaining tokens can be changed.


In [0]:
from six import iteritems
from smart_open import smart_open

# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in smart_open('mycorpus.txt', 'rb'))

# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)
print(dictionary)

2018-05-12 06:23:23,007 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-12 06:23:23,014 : INFO : built Dictionary(42 unique tokens: [u'and', u'minors', u'generation', u'testing', u'iv']...) from 9 documents (total 69 corpus positions)


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


And that is all there is to it! At least as far as bag-of-words representation is concerned. Of course, what we do with such a corpus is another question; it is not at all clear how counting the frequency of distinct words could be useful. As it turns out, it isn’t, and we will need to apply a transformation on this simple representation first, before we can use it to compute any meaningful document vs. document similarities. Transformations are covered in the [next tutorial](https://radimrehurek.com/gensim/tut2.html), but before that, let’s briefly turn our attention to *corpus persistency*.

## Corpus Formats

There exist several file formats for serializing a Vector Space corpus (~sequence of vectors) to disk. *Gensim* implements them via the *streaming corpus interface* mentioned earlier: documents are read from (or stored to) disk in a lazy fashion, one document at a time, without the whole corpus being read into main memory at once.

One of the more notable file formats is the [Matrix Market format](http://math.nist.gov/MatrixMarket/formats.html). To save a corpus in the Matrix Market format:

In [0]:
# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.mm'), corpus)

2018-05-12 06:23:24,155 : INFO : storing corpus in Matrix Market format to /tmp/corpus.mm
2018-05-12 06:23:24,158 : INFO : saving sparse matrix to /tmp/corpus.mm
2018-05-12 06:23:24,162 : INFO : PROGRESS: saving document #0
2018-05-12 06:23:24,164 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2018-05-12 06:23:24,166 : INFO : saving MmCorpus index to /tmp/corpus.mm.index


Other formats include [Joachim’s SVMlight format](http://svmlight.joachims.org/), [Blei’s LDA-C format](http://www.cs.columbia.edu/~blei/lda-c/) and [GibbsLDA++ format](http://gibbslda.sourceforge.net/).

In [0]:
corpora.SvmLightCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.svmlight'), corpus)
corpora.BleiCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.lda-c'), corpus)
corpora.LowCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.low'), corpus)

2018-05-12 06:23:25,253 : INFO : converting corpus to SVMlight format: /tmp/corpus.svmlight
2018-05-12 06:23:25,255 : INFO : saving SvmLightCorpus index to /tmp/corpus.svmlight.index
2018-05-12 06:23:25,258 : INFO : no word id mapping provided; initializing from corpus
2018-05-12 06:23:25,260 : INFO : storing corpus in Blei's LDA-C format into /tmp/corpus.lda-c
2018-05-12 06:23:25,262 : INFO : saving vocabulary of 2 words to /tmp/corpus.lda-c.vocab
2018-05-12 06:23:25,264 : INFO : saving BleiCorpus index to /tmp/corpus.lda-c.index
2018-05-12 06:23:25,266 : INFO : no word id mapping provided; initializing from corpus
2018-05-12 06:23:25,267 : INFO : storing corpus in List-Of-Words format into /tmp/corpus.low
2018-05-12 06:23:25,269 : WARNING : List-of-words format can only save vectors with integer elements; 1 float entries were truncated to integer value
2018-05-12 06:23:25,273 : INFO : saving LowCorpus index to /tmp/corpus.low.index


Conversely, to load a corpus iterator from a Matrix Market file:

In [0]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'corpus.mm'))

2018-05-12 06:23:26,359 : INFO : loaded corpus index from /tmp/corpus.mm.index
2018-05-12 06:23:26,362 : INFO : initializing cython corpus reader from /tmp/corpus.mm
2018-05-12 06:23:26,365 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


Corpus objects are streams, so typically you won’t be able to print them directly:

In [0]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)


Instead, to view the contents of a corpus:

In [0]:
# one way of printing a corpus: load it entirely into memory
print(list(corpus))  # calling list() will convert any sequence to a plain Python list

[[(1, 0.5)], []]


or

In [0]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

[(1, 0.5)]
[]


The second way is obviously more memory-friendly, but for testing and development purposes, nothing beats the simplicity of calling `list(corpus)`.

To save the same Matrix Market document stream in Blei’s LDA-C format,

In [0]:
corpora.BleiCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.lda-c'), corpus)

2018-05-12 06:23:30,510 : INFO : no word id mapping provided; initializing from corpus
2018-05-12 06:23:30,512 : INFO : storing corpus in Blei's LDA-C format into /tmp/corpus.lda-c
2018-05-12 06:23:30,513 : INFO : saving vocabulary of 2 words to /tmp/corpus.lda-c.vocab
2018-05-12 06:23:30,515 : INFO : saving BleiCorpus index to /tmp/corpus.lda-c.index


In this way, *gensim* can also be used as a memory-efficient **I/O format conversion tool**: just load a document stream using one format and immediately save it in another format. Adding new formats is dead easy, check out the [code for the SVMlight corpus](https://github.com/piskvorky/gensim/blob/develop/gensim/corpora/svmlightcorpus.py) for an example.

## Compatibility with NumPy and SciPy

Gensim also contains [efficient utility functions](http://radimrehurek.com/gensim/matutils.html) to help converting from/to `numpy` matrices:

In [0]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix_dense = gensim.matutils.corpus2dense(corpus, num_terms=10)

numpy_matrix

array([[1, 1],
       [2, 1],
       [0, 1],
       [9, 0],
       [5, 2]])

and from/to `scipy.sparse` matrices:

In [0]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)

scipy_sparse_matrix.todense()

matrix([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

For a complete reference (want to prune the dictionary to a smaller size? Optimize converting between corpora and NumPy/SciPy arrays?), see the [API documentation](https://radimrehurek.com/gensim/apiref.html). Or continue to the next tutorial on Topics and Transformations ([notebook](Topics_and_Transformations.ipynb) 
or [website](https://radimrehurek.com/gensim/tut2.html)).